In [ ]:
using NamedColors
using Colors
using ColorTypes
using StatsBase
using Learn
using MLDataUtils
using Iterators
using MultivariateStats

In [ ]:
const allowed_chars = Set(['a':'z'; '\''; ' '; '/'])

In [ ]:
mega_dict = merge(#load_crayola(),
                  #    load_resene(),
                      load_nbs(),
                      load_xkcd(),
                      load_x11())
test_training_dict=similar(mega_dict)
for (kk, vv) in mega_dict
    name = lowercase(kk)
    length(setdiff( Set(name), allowed_chars)) > 0  && continue
    test_training_dict[name] = vv
end
test_training_dict

In [ ]:
function encode_output(raw::Colorant)
    #Encoding Angles: http://stats.stackexchange.com/a/218547/36769
    hsv = convert(HSV, raw)
    [(sind(hsv.h)+1)/2, (cosd(hsv.h)+1)/2, hsv.s, hsv.v]
end

function decode_output(out)
    hue = rad2deg(atan2(2out[1]-1, 2out[2]-1))
    sat = out[3]
    val = out[4]
    HSV(hue, sat, val)
end


In [ ]:
const nchars = 26+2
function char2index(c::Char)::Int
    if c ∈ 'a':'z'
        c-'a'
    elseif c==' '
        'z'+1
    elseif c=='/'
        'z'+2
    elseif c=='\''
        'z'+3
    else
        error("$c is out of range")
    end
end

"""
Builds a ngram_encoder, out of a alphabet encoder
`n` the length of the ngrams
`m` the length of the alphabet
`alpha_encoder` must map the alphabet to the range 0:alphasize-1
"""
function ngram_encoder(alpha_encoder, alphasize)
    function ngram_encoder_inner(alphastring)
        n=length(alphastring)
        n_ngrams = alphasize^n-1
        alphasize.^(0:n-1)⋅alpha_encoder.(collect(alphastring))
    end
end

function encode_input(name::AbstractString; n=1)
    alphasize = 26+3
    coder = ngram_encoder(char2index, alphasize)
    
    ngrams = partition(name, n, 1) |> collect
    name_inds::Vector{Int} = coder.(ngrams)
    counts(name_inds, alphasize^n)
end

In [25]:
function get_data(data_dict)

    colors = collect(values(data_dict))
    ys = hcat(encode_output.(colors)...)
    
    names = collect(keys(data_dict))
    xs = hcat(encode_input.(names)...).*1.0

    #These could all be views, but PCA don't like that
    x_train = xs[:, 1:9*end÷10]
    y_train = ys[:, 1:9*end÷10]
    x_test = xs[:,1+9*end÷10 : end]
    y_test = ys[:,1+9*end÷10 : end]
    
    (x_train, y_train), (x_test, y_test)
end


get_data (generic function with 1 method)

In [26]:
function full_encode_input(M, name::AbstractString)
    x=encode_input(name)*1.0
    MultivariateStats.transform(M,x)
end

full_encode_input (generic function with 2 methods)

In [30]:
(x_train, y_train), (x_test, y_test) = get_data(test_training_dict)
M = fit(ICA, collect(x_train), 10)#; maxoutdim=100)



x_train_dr = MultivariateStats.transform(M,x_train)
x_test_dr =  MultivariateStats.transform(M,x_test)
(train,test) = (x_train_dr, y_train), (x_test_dr, y_test)

((
[1.45161 -0.152558 … 0.806574 -1.3548; 0.373618 -0.548537 … -0.429015 0.0241556; … ; -0.231092 0.191179 … 1.32662 -0.068711; 0.778269 0.0438234 … 0.250988 -0.871217],

Float32[0.994721 0.350743 … 0.618108 0.204548; 0.427534 0.977203 … 0.98585 0.0966291; 0.929032 0.892019 … 0.396078 0.857708; 0.607843 0.835294 … 1.0 0.992157]),(
[-0.917094 -0.828917 … 0.448083 1.17184; 0.759519 0.367389 … 0.76142 0.136729; … ; 0.766422 0.533557 … -0.794117 -0.429164; 1.11019 -0.34358 … -0.37434 0.944343],

Float32[0.711309 0.888573 … 0.230551 0.995183; 0.953154 0.81466 … 0.0788147 0.430766; 0.863309 0.144928 … 0.958333 0.595142; 0.545098 0.541176 … 0.376471 0.968627]))

In [31]:
function my_test_loss(obj, testdata)
    totloss = 0.0
    totcorrect = 0
    for (x,y) in eachobs(testdata)
        totloss += transform!(obj,y,x)
    end
    totloss#, totcorrect/totcount
end


tracer = IterFunction() do obj, i
    if mod1(i,1000)==1000
        totloss  = my_test_loss(obj, test)
        println(i, " totloss:  ", totloss)
        #println(i, " accuracy: ", accuracy)
    end
end


StochasticOptimization.IterFunction(#9)

In [32]:
nin, nh, nout = size(train[1],1), [100,50], size(train[2],1)
t = nnet(nin, nout, nh, :softplus, :logistic)
obj = objective(t, ElasticNetPenalty(1e-5))

ObjectiveFunctions.RegularizedObjective{Transformations.Chain{Float64,Transformations.Params{SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true},Tuple{},Tuple{}}},ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss},Penalties.ElasticNetPenalty{Float64}}(Chain{Float64}(
   Affine{10-->100}
   softplus{100}
   Affine{100-->50}
   softplus{50}
   Affine{50-->4}
   logistic{4}
) ,ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss}(LossFunctions.CrossentropyLoss(),4,input(4,),target(4,),output(1,)),Penalties.ElasticNetPenalty{Float64}(1.0e-5,0.5))

In [33]:
learner = make_learner(
    GradientLearner(5e-2, Adadelta()),
    tracer,
    maxiter = 50_000
)
learn!(obj, learner, infinite_batches(train, size=5))

1000 totloss:  307.2286445247243
2000 totloss:  302.4062560267238
3000 totloss:  300.62796138139254
4000 totloss:  299.94444006055863
5000 totloss:  297.26565633344495
6000 totloss:  297.5546861622681


LoadError: InterruptException:

In [ ]:
full_encode_input(M,"hyperblue")

In [38]:
function get_colour(obj, M, name)
    x = full_encode_input(M,name)
    decode_output( output_value(obj.transformation ))
end
get_colour(name) = get_colour(obj, M, name)


function display_colour(obj, M,name)
    display(name)
    display(get_colour(obj, M, name))
    display("-"^16)
end
display_colour(name)=display_colour(obj, M,name)

display_colour (generic function with 2 methods)

In [39]:
display_colour("green")
display_colour("red")
display_colour("lightred")
display_colour("dirtyred")
display_colour("lightredishblue")
display_colour("bluish red")
display_colour("blue")
display_colour("light blue")
display_colour("lightred")
 


"green"

"----------------"

"red"

"----------------"

"lightred"

...

"----------------"

"light blue"

"----------------"

"lightred"

"----------------"

In [40]:
display_colour("hot pink")
display_colour("hot blue")
display_colour("hot green")
display_colour("dark blue")
display_colour("light black")
display_colour("black")
display_colour("dark black")

"hot pink"

"----------------"

"hot blue"

"----------------"

"hot green"

...

"----------------"

"black"

"----------------"

"dark black"

"----------------"

In [41]:
for letter in ['a':'z'; ' '; '\''; '/']
    display_colour(obj, string(letter))
end

LoadError: MethodError: no method matching display_colour(::ObjectiveFunctions.RegularizedObjective{Transformations.Chain{Float64,Transformations.Params{SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true},Tuple{},Tuple{}}},ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss},Penalties.ElasticNetPenalty{Float64}}, ::String)[0m
Closest candidates are:
  display_colour(::Any, ::Any, [1m[31m::Any[0m) at In[38]:9
  display_colour(::Any) at In[38]:13[0m

In [24]:
for letter1 in ['a':'z'; ' '; '\''; '/']
    for letter2 in ['a':'z'; ' '; '\''; '/']
        display_colour(obj, join([letter1,letter2]))
    end
end

LoadError: MethodError: no method matching display_colour(::ObjectiveFunctions.RegularizedObjective{Transformations.Chain{Float64,Transformations.Params{SubArray{Float64,1,Array{Float64,1},Tuple{UnitRange{Int64}},true},Tuple{},Tuple{}}},ObjectiveFunctions.LossTransform{Float64,LossFunctions.CrossentropyLoss},Penalties.ElasticNetPenalty{Float64}}, ::String)[0m
Closest candidates are:
  display_colour(::Any, ::Any, [1m[31m::Any[0m) at In[19]:9
  display_colour(::Any) at In[19]:13[0m

In [ ]:
1+1